In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
trainMP=pd.read_csv(r'C:\Users\Nelson\Documents\Sync\Archivos de práctica Curso\trainMF.csv')# este archivo esta bien
trainMP=trainMP.drop('Unnamed: 0',axis=1)
trainMP

,Survived,Pclass,SibSp,Parch,Fare,Sex Encode,Embarked-2,Age-sep
0,0,3,1,0,7.250,1,2,1
1,1,1,1,0,712.833,0,0,3
2,1,3,0,0,7.925,0,2,2
3,1,1,1,0,53.100,0,2,3
4,0,3,0,0,8.050,1,2,3
...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.000,1,2,2
887,1,1,0,0,30.000,0,2,1
888,0,3,1,2,23.450,0,2,2
889,1,1,0,0,30.000,1,0,2


In [3]:
trainME=trainMP.drop('Survived', axis=1)
y=trainMP['Survived'] 
X=trainME

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_test

,Pclass,SibSp,Parch,Fare,Sex Encode,Embarked-2,Age-sep
709,3,1,1,152.458,1,0,2
439,2,0,0,10.500,1,2,2
840,3,0,0,7.925,1,2,1
720,2,0,1,33.000,0,2,0
39,3,1,0,112.417,0,0,1
...,...,...,...,...,...,...,...
494,3,0,0,8.050,1,2,1
359,3,0,0,78.792,0,1,2
65,3,1,1,152.458,1,0,2
481,2,0,0,0.000,1,2,2


In [6]:
#Out of Bag
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier
param_grid = ParameterGrid(
                {'n_estimators': [150],
                 'max_features': [2,4, 5, 7],
                 'max_depth'   : [None, 3, 10, 20],
                 'criterion'   : ['gini', 'entropy']
                }
            )

# Loop para ajustar un modelo con cada combinación de hiperparámetros

resultados = {'params': [], 'oob_accuracy': []}

for params in param_grid:
    
    modelo1 = RandomForestClassifier(
                oob_score    = True,
                n_jobs       = -1,
                random_state = 42,
                ** params
             )
    
    modelo1.fit(X_train, y_train)
    
    resultados['params'].append(params)
    resultados['oob_accuracy'].append(modelo1.oob_score_)
    print(f"Modelo: {params} \u2713")

resultados = pd.DataFrame(resultados)
resultados = pd.concat([resultados, resultados['params'].apply(pd.Series)], axis=1)
resultados = resultados.sort_values('oob_accuracy', ascending=False)
resultados = resultados.drop(columns = 'params')
resultados.head(4)

Modelo: {'criterion': 'gini', 'max_depth': None, 'max_features': 2, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': None, 'max_features': 4, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': None, 'max_features': 5, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': None, 'max_features': 7, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': 3, 'max_features': 2, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': 3, 'max_features': 4, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': 3, 'max_features': 5, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': 3, 'max_features': 7, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': 10, 'max_features': 2, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': 10, 'max_features': 4, 'n_estimators': 150} ✓
Modelo: {'criterion': 'gini', 'max_depth': 10, 'max_features': 5, 'n_estimators': 150} ✓
Modelo: {'criteri

,oob_accuracy,criterion,max_depth,max_features,n_estimators
25,0.817701,entropy,10.0,4,150
26,0.815059,entropy,10.0,5,150
11,0.813738,gini,10.0,7,150
7,0.812417,gini,3.0,7,150


In [8]:
#Grid Search Muuucho mas lentoque oob y algo de menos precisión
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [150],
              'max_features': [2,4,5, 7],
              'max_depth'   : [None, 3, 10, 20],
              'criterion'   : ['gini', 'entropy']
             }

# Búsqueda por grid search con validación cruzada

grid = GridSearchCV(
        estimator  = RandomForestClassifier(random_state = 42),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X_train, y = y_train)


resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

,param_criterion,param_max_depth,param_max_features,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
24,entropy,10,2,150,0.815482,0.025203,0.931308,0.005619
26,entropy,10,5,150,0.813718,0.025070,0.932409,0.003979
25,entropy,10,4,150,0.812841,0.020924,0.931968,0.005334
27,entropy,10,7,150,0.811511,0.028215,0.935271,0.005757


In [9]:
clasificadorF=RandomForestClassifier(n_estimators=150,  criterion='entropy', max_depth=10, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=True, n_jobs=-1, random_state=42, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

In [11]:
clasificadorF.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, max_features=4,
                       n_estimators=150, n_jobs=-1, oob_score=True,
                       random_state=42)

In [12]:
clasificadorF.score(X_train, y_train)

0.9299867899603699

In [13]:
clasificadorF.score(X_test, y_test)#Subió 3 puntos respecto de tree y es de las más altas.Parece overfitting?

0.8656716417910447

In [16]:
predicciones = clasificadorF.predict(X = X_test)

In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
mat_confusion = confusion_matrix(y_true= y_test,y_pred=predicciones)
print(mat_confusion)
print(classification_report(y_true = y_test,y_pred = predicciones))

[[71  7]
 [11 45]]
              precision    recall  f1-score   support

           0       0.87      0.91      0.89        78
           1       0.87      0.80      0.83        56

    accuracy                           0.87       134
   macro avg       0.87      0.86      0.86       134
weighted avg       0.87      0.87      0.86       134



In [19]:
importancia_predictores = pd.DataFrame(
                            {'predictor': X_train.columns,
                             'importancia': clasificadorF.feature_importances_}
                            )
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores.sort_values('importancia', ascending=False)

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
4,Sex Encode,0.305648
3,Fare,0.285499
0,Pclass,0.131577
6,Age-sep,0.120862
1,SibSp,0.075297
2,Parch,0.047274
5,Embarked-2,0.033843
